# Материалы

- [Reuven M. Lerner - Generators, coroutines, and nanoservices || PyCon Africa 2020](https://youtu.be/tkoaeVS2zRQ)
- https://realpython.com/introduction-to-python-generators/
- [Curious Course on Coroutines and Concurrency (David Beazley)](https://youtu.be/Z_OAlIhXziw)
- http://dabeaz.com/coroutines/Coroutines.pdf
- https://stackoverflow.com/questions/9708902/in-practice-what-are-the-main-uses-for-the-yield-from-syntax-in-python-3-3
- https://stackoverflow.com/questions/35518986/whats-the-difference-between-yield-from-and-yield-in-python-3-3-2

Моя аналогия

Грузовичок выезжает без груза (если `g = gen()`, то возможны только `g.send(None)` или `next(g)`), по дороге собирает груз.

На 1-ой `yield`-остановке разгружает груз. Ждет загрузки (с помощью `send`).

## Функция

In [1]:
def myfunc():
    return 1
    return 2
    return 3

In [2]:
myfunc

<function __main__.myfunc()>

In [3]:
import dis

# Even Python compiler ignores return 2 return 3
dis.dis(myfunc)  # => bytecodes from our function

  2           0 LOAD_CONST               1 (1)
              2 RETURN_VALUE


## Генератор (generator)

`yield` turns function into `generator function`. When Python compiles your function it notices `yield` and tags the function as a `generator function`.

generator_function() => generator (object)

Calling a generator function creates an generator object. However, it does not start running the function.

`yield` produces a value, but suspends the function

In [4]:
def mygen():
    print('Грузовичк выехал')
    yield 1  # Прибыл в 1-ый yield-пункт, выгрузил 1, ждет next/send
    yield 2
    yield 3

In [5]:
g = mygen()
g2 = mygen()

In [6]:
next(g)

Грузовичк выехал


1

In [7]:
next(g)

2

In [8]:
next(g2)

Грузовичк выехал


1

In [9]:
next(g)

3

In [10]:
# next(g)  # => StopIteration

In [11]:
dis.show_code(mygen)  # Notice the flag GENERATOR!

Name:              mygen
Filename:          /tmp/ipykernel_107508/3303880497.py
Argument count:    0
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  0
Stack size:        2
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR, NOFREE
Constants:
   0: None
   1: 'Грузовичк выехал'
   2: 1
   3: 2
   4: 3
Names:
   0: print


In [12]:
dis.dis(mygen)  # три YIELD_VALUE (нет игнорирования!)

              0 GEN_START                0

  2           2 LOAD_GLOBAL              0 (print)
              4 LOAD_CONST               1 ('Грузовичк выехал')
              6 CALL_FUNCTION            1
              8 POP_TOP

  3          10 LOAD_CONST               2 (1)
             12 YIELD_VALUE
             14 POP_TOP

  4          16 LOAD_CONST               3 (2)
             18 YIELD_VALUE
             20 POP_TOP

  5          22 LOAD_CONST               4 (3)
             24 YIELD_VALUE
             26 POP_TOP
             28 LOAD_CONST               0 (None)
             30 RETURN_VALUE


## Generator expression/comprehension

You can also define a `generator expression` (also called a `generator comprehension`),
which has a very similar syntax to list comprehensions.

Remember, list comprehensions return full lists, while generator expressions return generators.
Generators work the same whether they’re built from a function or an expression.
Using an expression just allows you to define simple generators in a single line,
with an assumed yield at the end of each inner iteration.

In [13]:
# In this way, you can use the generator without calling a function:
# csv_gen = (row for row in open(file_name))

## Iterator protocol

In [14]:
for i in 'abc':
    print(i)

a
b
c


In [15]:
for i in mygen():
    print(i)

Грузовичк выехал
1
2
3


In [16]:
g = mygen()
g

<generator object mygen at 0x7fec9c27af80>

In [17]:
iter(g)

<generator object mygen at 0x7fec9c27af80>

In [18]:
g is iter(g)

True

In [19]:
next(g), next(g), next(g)

Грузовичк выехал


(1, 2, 3)

In [20]:
try:
    next(g)
except StopIteration as e:
    print(type(e))

<class 'StopIteration'>


## Generators as Pipelines

- One of the most powerful applications of generators is setting up processing pipelines
- Similar to shell pipes in Unix
- Idea: You can stack a series of generator functions together into a pipe and pull items through it with a for-loop

`input sequence -> [generator] -> [generator] -> [generator] -> for x in s:`

In [21]:
def even_filter(nums):
    for num in nums:
        if num % 2 == 0:
            yield num

def multiply_by_three(nums):
    for num in nums:
        yield num * 3

def convert_to_string(nums):
    for num in nums:
        yield f'The Number: {num}'

In [22]:
nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
pipeline = convert_to_string(multiply_by_three(even_filter(nums)))
for num in pipeline:
    print(num)

The Number: 6
The Number: 12
The Number: 18
The Number: 24
The Number: 30


In [23]:
# file_name = 'techcrunch.csv'
# lines = (line for line in open(file_name))
# list_line = (s.rstrip().split(',') for s in lines)
# cols = next(list_line)
# company_dicts = (dict(zip(cols, data)) for data in list_line)
# funding = (
#     int(company_dict['raisedAmt'])
#     for company_dict in company_dicts
#     if company_dict['round'] == 'a'
# )
# total_series_a = sum(funding)
# print(f'Total series A fundraising: ${total_series_a}')

In [24]:
# мой пример
def gf1():
    yield 1
    yield 2
    yield 3

def gf2(g):
    for n in g:
        if n % 2:
            yield n

list(gf2(gf1()))

[1, 3]

In [25]:
list(gf2(x for x in [7, 8, 9, 10]))

[7, 9]

In [26]:
a = (i for i in range(10))
b = (i*i for i in a if i % 2)
c = (f'-{i}-' for i in b)
list(c)

['-1-', '-9-', '-25-', '-49-', '-81-']

## Coroutine

In Python 2.5, a slight modification to the yield statement was introduced (PEP-342). You could now use yield as an expression. If you use yield more generally, you get a coroutine.

In [27]:
def mygen():
    x = None
    while True:
        x = yield x
        x *= 5

In [28]:
g = mygen()

In [29]:
next(g)  # priming, same as g.send(None)

In [30]:
g.send(5)

25

In [31]:
g.send(10)

50

In [32]:
g.send('abc')

'abcabcabcabcabc'

In [33]:
g.send([1, 2, 3])

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]

### Walrus + yield

In [34]:
def mygen():
    x = None
    while x := (yield x):
        x *= 5

In [35]:
g = mygen()
g.send(None)
g.send(2)

10

### Coroutine Priming

У David Beazley `@coroutine` вместо `@prime`.

In [36]:
from functools import wraps

# Грузовичок срывается из гаража, доезжает до первой yield-остановки,
# груз выгружает (он при этом игнорируется) и
# ждет нового груза (он будет загружен через `send`).
def prime(generator_func):
    @wraps(generator_func)
    def inner(*args, **kwargs):
        generator = generator_func(*args, **kwargs)
        next(generator)
        return generator
    return inner

### Coroutine as a nanoservices

In [37]:
# https://worldweather.wmo.int/en/dataguide.html
# "Russian Federation";"Moscow";"206"
import requests

class DifferentCityException(Exception):
    pass

def get_forecasts():
    while city_id := (yield 'Enter city id (Moscow id is 206):'):
        weather = requests.get(f'https://worldweather.wmo.int/en/json/{city_id}_en.json').json()
        try:
            for one_forecast in weather['city']['forecast']['forecastDay']:
                yield one_forecast
        except DifferentCityException:
            continue

In [38]:
g = get_forecasts()
next(g)

'Enter city id (Moscow id is 206):'

In [39]:
g.send(44)

{'forecastDate': '2022-09-26',
 'wxdesc': '',
 'weather': 'Clear',
 'minTemp': '22',
 'maxTemp': '30',
 'minTempF': '72',
 'maxTempF': '86',
 'weatherIcon': 2502}

In [40]:
g.send(None)  # повторить

{'forecastDate': '2022-09-27',
 'wxdesc': '',
 'weather': 'Clear',
 'minTemp': '22',
 'maxTemp': '30',
 'minTempF': '72',
 'maxTempF': '86',
 'weatherIcon': 2502}

In [41]:
g.throw(DifferentCityException)

'Enter city id (Moscow id is 206):'

In [42]:
g.send(206)

{'forecastDate': '2022-09-27',
 'wxdesc': '',
 'weather': 'Fine',
 'minTemp': '3',
 'maxTemp': '12',
 'minTempF': '37',
 'maxTempF': '54',
 'weatherIcon': 2501}

In [43]:
g.send(None)

{'forecastDate': '2022-09-28',
 'wxdesc': '',
 'weather': 'Rain',
 'minTemp': '7',
 'maxTemp': '10',
 'minTempF': '45',
 'maxTempF': '50',
 'weatherIcon': 1401}

### Closing a Coroutine

- A coroutine might run indefinitely
- Use `.close()` to shut it down
- Note: Garbage collection also calls `close()`
- `close()` can be caught (GeneratorExit)
- You cannot ignore this exception
- Only legal action is to clean up and return

- https://amir.rachum.com/blog/2017/03/03/generator-cleanup/
- https://docs.python.org/3/reference/expressions.html#generator.close
- https://stackoverflow.com/questions/60137570/explanation-of-generator-close-with-exception-handling

Closing is like throwing GeneratorExit exception

In [44]:
def gen():
    yield '1'
    yield '2'
    yield '3'

In [45]:
# исключение GeneratorExit при выходе исчезает
g = gen()
next(g)
g.close()

In [46]:
# already closed
g.close()

In [47]:
# If the generator yields a value, a RuntimeError is raised
def gen():
    print('to 1')
    yield '1'
    try:
        print('to 2')
        yield '2'
    except GeneratorExit:
        print('to 3')
        yield '3'
    yield 'bye'

g = gen()
print(next(g))
print(next(g))
# g.close() => RuntimeError

to 1
1
to 2
2


In [48]:
# If the generator raises any other exception, it is propagated to the caller
def gen():
    print('to 1')
    yield '1'
    try:
        print('to 2')
        yield '2'
    except GeneratorExit:
        print('to 3')
        raise ValueError('value error')
    yield 'bye'

g = gen()
print(next(g))
print(next(g))
try:
   g.close()
except ValueError as e:
    print(e)

Exception ignored in: <generator object gen at 0x7fec9cb75540>
Traceback (most recent call last):
  File "/tmp/ipykernel_107508/221717615.py", line 13, in <cell line: 13>
RuntimeError: generator ignored GeneratorExit


to 3
to 1
1
to 2
2
to 3
value error


### Throwing an Exception

- Exceptions can be thrown inside a coroutine
- Exception originates at the yield expression
- Can be caught/handled in the usual ways

Аналогично send, но только исключения. Грузовичок едет дальше!

In [49]:
def gen():
    print('to 1')
    yield '1'
    try:
        print('to 2')
        yield '2'
    except ValueError:
        print('to 3')
        yield '3'
    yield 'bye'

In [50]:
g = gen()
print(next(g))
print(next(g))
print(g.throw(ValueError('error')))  # грузовичок поехал дальше!
print(next(g))

to 1
1
to 2
2
to 3
3
bye


### Coroutine vs generator

David Beazley:

Despite some similarities, generators and coroutines are basically two different concepts:

- Generators produce values
- Coroutines tend to consume values
- It is easy to get sidetracked because methods meant for coroutines are sometimes described as
  a way to tweak generators that are in the process of producing an iteration pattern (i.e., resetting its
  value). This is mostly bogus. See example below.

Keeping it straight:

- Generators produce data for iteration
- Coroutines are consumers of data
- To keep your brain from exploding, you don't mix the two concepts together
- Coroutines are not related to iteration
- Note: There is a use of having yield produce a value in a coroutine, but it's not tied to iteration.

In [51]:
# bogus example
def countdown(n):
    print('Counting down from', n)
    while n >= 0:
        newvalue = (yield n)
        # If a new value got sent in, reset n with it
        if newvalue is not None:
            n = newvalue
        else:
            n -= 1

c = countdown(5)
for n in c:
    print(n)
    if n == 5:
        c.send(3)

Counting down from 5
5
2
1
0


In [52]:
c = countdown(5)
for n in c:
    print(n)
    if n == 5:
        print(c.send(3))  # здесь 3 было "потеряно"

Counting down from 5
5
3
2
1
0


## yield from

What yield from does is it establishes a `transparent bidirectional connection` between the caller and the sub-generator:

- The connection is "transparent" in the sense that it will propagate everything correctly too, not just the elements being generated (e.g. exceptions are propagated).

- The connection is "bidirectional" in the sense that data can be both sent from and to a generator.

## Мой пример

In [53]:
class LevelUpExit(Exception):
    pass

In [54]:
# def gen1():
#     """gen1"""
#     x = 'gen1 help'
#     while x := (yield x):
#         x = f'gen1({x})'

def gen1():
    """gen1"""
    x = 'gen1 help'
    while True:
        try:
            x = yield x
        except LevelUpExit:
            break
        x = f'gen1({x})'

In [55]:
def gen2():
    """gen2"""
    x = 'gen2 help'
    while True:
        try:
            x = yield x
        except LevelUpExit:
            break
        x = f'gen2({x})'

In [56]:
def combined(gen_functions):
    G = {str(n): gf for n, gf in enumerate(gen_functions, start=1)}
    welcome = '\n'.join(f'{n}: {gf.__doc__}' for n, gf in G.items())
    welcome += '\nq: quit'
    message = welcome
    try:
        while n := (yield message):
            if n in G:
                yield from G[n]()
                message = welcome
            else:
                message = f'Выбор неверен. Повторите.\n{welcome}'
    except GeneratorExit:
        print('cleanup')

In [57]:
g = combined([gen1, gen2])

In [58]:
g.send(None)

'1: gen1\n2: gen2\nq: quit'

In [59]:
g.send('1')

'gen1 help'

In [60]:
g.send(20)

'gen1(20)'

In [61]:
g.throw(LevelUpExit)

'1: gen1\n2: gen2\nq: quit'

In [62]:
g.send('2')

'gen2 help'

In [63]:
g.send(7)

'gen2(7)'

In [64]:
g.throw(LevelUpExit)

'1: gen1\n2: gen2\nq: quit'

In [65]:
# g.throw(LevelUpExit) => LevelUpExit

In [67]:
DISABLED = True
g = combined([gen1, gen2])
output_str = next(g)
while not DISABLED:
    print(output_str)
    input_str = input(output_str)
    if input_str.lower() == 'q':
        try:
            outputs_str = g.throw(LevelUpExit)
            pass
        except LevelUpExit:
            break
    else:
        output_str = g.send(input_str)

cleanup


## Type hinting

https://mypy.readthedocs.io/en/stable/kinds_of_types.html#generators